In [ ]:
%%bash
gcloud auth list

gcloud is the powerful and unified command-line tool for Google Cloud Platform

In [ ]:
%%bash
gcloud config list project

Once connected to the cloud shell, you should see that you are already authenticated and that the project is already set to your PROJECT_ID.

List the Cloud ML models to verify that the command returns an empty list:

In [ ]:
%%bash
gcloud ml-engine models list

Clone the GitHub repository:


In [ ]:
%%bash
git clone https://github.com/GoogleCloudPlatform/cloudml-dist-mnist-example
cd cloudml-dist-mnist-example

Next you'll create a Cloud Storage bucket to hold the MNIST data files that are used to train the model.

In [ ]:
%%bash
##Paste below code in cloud console
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
MNIST_BUCKET="${PROJECT_ID}-ml"
gsutil mb -c regional -l us-east1 gs://${MNIST_BUCKET}

Use the following script to download the MNIST data files and copy them to the bucket:

In [ ]:
%%bash
##Paste below code in cloud console
sudo ./scripts/create_records.py
gsutil cp /tmp/data/train.tfrecords gs://${MNIST_BUCKET}/data/
gsutil cp /tmp/data/test.tfrecords gs://${MNIST_BUCKET}/data/

Next you'll create a Cloud Storage bucket to hold the MNIST data files that are used to train the model.

In [ ]:
%%bash
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
MNIST_BUCKET="${PROJECT_ID}-ml"
gsutil mb -c regional -l us-east1 gs://${MNIST_BUCKET}

Use the following script to download the MNIST data files and copy them to the bucket:

In [ ]:
%%bash
sudo ./scripts/create_records.py
gsutil cp /tmp/data/train.tfrecords gs://${MNIST_BUCKET}/data/
gsutil cp /tmp/data/test.tfrecords gs://${MNIST_BUCKET}/data/


Submit a training job to Cloud ML Engine.

In [ ]:
%%bash
JOB_NAME="job_$(date +%Y%m%d_%H%M%S)"
gcloud ml-engine jobs submit training ${JOB_NAME} \
    --package-path trainer \
    --module-name trainer.task \
    --staging-bucket gs://${MNIST_BUCKET} \
    --job-dir gs://${MNIST_BUCKET}/${JOB_NAME} \
    --runtime-version 1.2 \
    --region us-central1 \
    --config config/config.yaml \
    -- \
    --data_dir gs://${MNIST_BUCKET}/data \
    --output_dir gs://${MNIST_BUCKET}/${JOB_NAME} \
    --train_steps 5000

Run the following command in Cloud Shell to start TensorBoard:

In [ ]:
tensorboard --port 8080 --logdir gs://${MNIST_BUCKET}/${JOB_NAME}

To open a new browser window, select Preview on port 8080 from the Web preview menu in the top-right corner of the Cloud Shell toolbar.

You deploy the trained model for predictions using the model binary.

Deploy the model and set the default version.

In [ ]:
%%bash
MODEL_NAME=MNIST
gcloud ml-engine models create --regions us-central1 ${MODEL_NAME}
VERSION_NAME=v1
ORIGIN=$(gsutil ls gs://${MNIST_BUCKET}/${JOB_NAME}/export/Servo | tail -1)
gcloud ml-engine versions create \
    --origin ${ORIGIN} \
    --runtime-version=1.2 \
    --model ${MODEL_NAME} \
    ${VERSION_NAME}

In [ ]:
%%bash
gcloud ml-engine versions set-default --model ${MODEL_NAME} ${VERSION_NAME}

These commands will take about five minutes to complete.

Test the prediction API by using a sample request file.

In [ ]:
%%bash
./scripts/make_request.py

This script creates a JSON file, named request.json, containing 10 test images for predictions.

Submit an online prediction request.

In [ ]:
%%bash
gcloud ml-engine predict --model ${MODEL_NAME} --json-instances request.json

Run this:

In [ ]:
%%bash
wget https://raw.githubusercontent.com/GoogleCloudPlatform/cloudml-dist-mnist-example/master/notebooks/Online%20prediction%20example.ipynb
cat Online\ prediction\ example.ipynb > Untitled\ Notebook.ipynb